<a href="https://colab.research.google.com/github/wudih146/PythonML/blob/main/testnlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os 
os.getcwd()
def load_dictionary():
    dic = set()

    # 按行读取字典文件，每行第一个空格之前的字符串提取出来。
    for line in open("CoreNatureDictionary.mini.txt","r"):
        dic.add(line[0:line.find('	')])
    
    return dic
dic=load_dictionary()
mydic=list(dic)
mydic[:100]

['记叙',
 '责任书',
 '高官',
 '空管局',
 '平平淡淡',
 '校时钟',
 '试穿',
 '宦',
 '半世',
 '启',
 '夤',
 '燕南园',
 '端电压',
 '打头',
 '突地',
 '绍兴市',
 '缨',
 '目眩',
 '髯口',
 '敷料',
 '梆子',
 '树形',
 '无以',
 '挺胸',
 '秒',
 '记要',
 '向心',
 '非农业',
 '恢宏',
 '委以',
 '下家',
 '沙鸡',
 '灵光',
 '万方',
 '宁肯',
 '螟蛉',
 '缝缝补补',
 '京山县',
 '言之凿凿',
 '标的',
 '充要条件',
 '垸',
 '磨合期',
 '绷子',
 '莘',
 '女巫',
 '人造板',
 '底肥',
 '招待',
 '收支簿',
 '苦处',
 '轻机关枪',
 '式微',
 '翻译器',
 '不好',
 '冬运',
 '北方邦',
 '橐',
 '后撤',
 '薄父母',
 '白食',
 '鼓吹者',
 '骛',
 '座上宾',
 '纾',
 '入土',
 '分析语',
 '土模',
 '源源',
 '阻抗',
 '肘子',
 '醉态',
 '大胜',
 '军工厂',
 '克尽职守',
 '纵身',
 '进货关',
 '送别',
 '邮电部',
 '拐',
 '农区',
 '胡乱',
 '棉籽饼',
 '小鬼',
 '拍卖行',
 '接访',
 '水温',
 '海涂',
 '科罗尔',
 '硅石',
 '统一战线',
 '全国妇联',
 '肩胛骨',
 '船速',
 '转引',
 '弄权',
 '试射',
 '豆油',
 '酱坊',
 '词汇表']

完全切分

In [2]:
def fully_segment(text, dic):
    word_list = []
    for i in range(len(text)):                  # i 从 0 到text的最后一个字的下标遍历
        for j in range(i + 1, len(text) + 1):   # j 遍历[i + 1, len(text)]区间
            word = text[i:j]                    # 取出连续区间[i, j]对应的字符串
            if word in dic:                     # 如果在词典中，则认为是一个词
                word_list.append(word)
    return word_list
  
dic = load_dictionary()
print(fully_segment('就读北京大学', dic))

['就', '就读', '读', '北', '北京', '北京大学', '京', '大', '大学', '学']


正向最长匹配

In [3]:
def forward_segment(text, dic):
    word_list = []
    i = 0
    while i < len(text):
        longest_word = text[i]                      # 当前扫描位置的单字
        for j in range(i + 1, len(text) + 1):       # 所有可能的结尾
            word = text[i:j]                        # 从当前位置到结尾的连续字符串
            if word in dic:                         # 在词典中
                if len(word) > len(longest_word):   # 并且更长
                    longest_word = word             # 则更优先输出
        word_list.append(longest_word)              # 输出最长词
        i += len(longest_word)                      # 正向扫描
    return word_list

dic = load_dictionary()
print(forward_segment('就读北京大学', dic))
print(forward_segment('研究生命起源', dic))

['就读', '北京大学']
['研究生', '命', '起源']


逆向最长匹配

In [4]:
def backward_segment(text, dic):
    word_list = []
    i = len(text) - 1
    while i >= 0:                                   # 扫描位置作为终点
        longest_word = text[i]                      # 扫描位置的单字
        for j in range(0, i):                       # 遍历[0, i]区间作为待查询词语的起点
            word = text[j: i + 1]                   # 取出[j, i]区间作为待查询单词
            if word in dic:
                if len(word) > len(longest_word):   # 越长优先级越高
                    longest_word = word
                    break
        word_list.insert(0, longest_word)           # 逆向扫描，所以越先查出的单词在位置上越靠后
        i -= len(longest_word)
    return word_list

dic = load_dictionary()
print(backward_segment('研究生命起源', dic))
print(backward_segment('项目的研究', dic))

['研究', '生命', '起源']
['项', '目的', '研究']


双向最长匹配

In [5]:
def count_single_char(word_list: list):  # 统计单字成词的个数
    return sum(1 for word in word_list if len(word) == 1)


def bidirectional_segment(text, dic):
    f = forward_segment(text, dic)
    b = backward_segment(text, dic)
    if len(f) < len(b):                                  # 词数更少优先级更高
        return f
    elif len(f) > len(b):
        return b
    else:
        if count_single_char(f) < count_single_char(b):  # 单字更少优先级更高
            return f
        else:
            return b                                     # 都相等时逆向匹配优先级更高
        

print(bidirectional_segment('研究生命起源', dic))
print(bidirectional_segment('项目的研究', dic))

['研究', '生命', '起源']
['项', '目的', '研究']


# 字典树与HanLP词典分词实现

In [6]:

## 节点类 
class Node():
    def __init__(self) -> None:
        self.children = {}
        self.value = None
    
    # 增加节点
    def add_child(self, char, value, overwrite=False):
        child = self.children.get(char)
        if child is None:
            child = Node()                # 创建子节点
            self.children[char] = child   # 子节点赋值，字 -> 节点的映射
        
        if value is not None or overwrite:
            child.value = value           # 节点上对应的词
        
        return child
    
## 字典树  继承节点类   
class Trie(Node):

    def __contains__(self, key):
        return self[key] is not None
    
    # 查询方法
    def __getitem__(self, key):
        state = self
        for char in key:
            state = state.children.get(char)
            if state is None:
                return None
        
        return state.value
    
    # 重载方法，使得类可以像对待dict那样操作字典树
    # 构建一个词的字典树
    def __setitem__(self, key, value):
        state = self
        for i, char in enumerate(key):
            if i < len(key) - 1:
                state = state.add_child(char, None)
            else:
                state = state.add_child(char, value, True)


if __name__ == '__main__':
    trie = Trie()
    # 增
    trie['自然'] = 'nature'
    trie['自然人'] = 'human'
    trie['自然语言'] = 'language'
    trie['自语'] = 'talk	to oneself'
    trie['入门'] = 'introduction'
    assert '自然' in trie
    # 删
    trie['自然'] = None
    assert '自然' not in trie
    # 改
    trie['自然语言'] = 'human language'
    assert trie['自然语言'] == 'human language'
    # 查
    assert trie['入门'] == 'introduction'
    print()


HanLP的词典分词实现

In [7]:
# pip install pyhanlp

     |████████████████████████████████| 92kB 7.8MB/s 
     |████████████████████████████████| 2.7MB 11.8MB/s 
  Created wheel for pyhanlp: filename=pyhanlp-0.1.66-py2.py3-none-any.whl size=29371 sha256=050aa3296b50fbe698d63a084b943b503b541d572ccc25a360e305d18d2f6159
  Stored in directory: /root/.cache/pip/wheels/25/8d/5d/6b642484b1abd87474914e6cf0d3f3a15d8f2653e15ff60f9e
Successfully built pyhanlp


In [22]:
from pyhanlp import *

# 不显示词性
HanLP.Config.ShowTermNature = False

# 可传入自定义字典 [dir1, dir2]
segment = DoubleArrayTrieSegment()
# 激活数字和英文识别
segment.enablePartOfSpeechTagging(True)

print(segment.seg("江西鄱阳湖干枯，中国最大淡水湖变成大草原"))
print(segment.seg("上海市虹口区大连西路550号SISU"))

java.lang.IndexOutOfBoundsException: ignored

去掉停用词

In [19]:
def load_from_file(path):
    """
    从词典文件加载DoubleArrayTrie
    :param path: 词典路径
    :return: 双数组trie树
    """
    map = JClass('java.util.TreeMap')()  # 创建TreeMap实例
    with open(path) as src:
        for word in src:
            word = word.strip()  # 去掉Python读入的\n
            map[word] = word
    return JClass('com.hankcs.hanlp.collection.trie.DoubleArrayTrie')(map)


## 去掉停用词
def remove_stopwords_termlist(termlist, trie):
    return [term.word for term in termlist if not trie.containsKey(term.word)]


trie = load_from_file('stopwords.txt')
termlist = segment.seg("江西鄱阳湖干枯了，中国最大的淡水湖变成了大草原")
print('去掉停用词前：', termlist)

print('去掉停用词后：', remove_stopwords_termlist(termlist, trie))

NameError: ignored

# 二元语法与中文分词

训练与预测

In [ ]:
from pyhanlp import *
from jpype import JString

## 加载 JAVA 类
CorpusLoader = SafeJClass('com.hankcs.hanlp.corpus.document.CorpusLoader')
NatureDictionaryMaker = SafeJClass('com.hankcs.hanlp.corpus.dictionary.NatureDictionaryMaker')
CoreDictionary = LazyLoadingJClass('com.hankcs.hanlp.dictionary.CoreDictionary')
WordNet = JClass('com.hankcs.hanlp.seg.common.WordNet')
Vertex = JClass('com.hankcs.hanlp.seg.common.Vertex')


def generate_wordnet(sent, trie):
    """
    生成词网
    :param sent: 句子
    :param trie: 词典（unigram）
    :return: 词网
    """
    searcher = trie.getSearcher(JString(sent), 0)
    wordnet = WordNet(sent)
    while searcher.next():
        wordnet.add(searcher.begin + 1,
                    Vertex(sent[searcher.begin:searcher.begin + searcher.length], searcher.value, searcher.index))
    # 原子分词，保证图连通
    vertexes = wordnet.getVertexes()
    i = 0
    while i < len(vertexes):
        if len(vertexes[i]) == 0:  # 空白行
            j = i + 1
            for j in range(i + 1, len(vertexes) - 1):  # 寻找第一个非空行 j
                if len(vertexes[j]):
                    break
            wordnet.add(i, Vertex.newPunctuationInstance(sent[i - 1: j - 1]))  # 填充[i, j)之间的空白行
            i = j
        else:
            i += len(vertexes[i][-1].realWord)

    return wordnet


## 维特比算法
def viterbi(wordnet):
    nodes = wordnet.getVertexes()
    # 前向遍历
    for i in range(0, len(nodes) - 1):
        for node in nodes[i]:
            for to in nodes[i + len(node.realWord)]:
                to.updateFrom(node)  # 根据距离公式计算节点距离，并维护最短路径上的前驱指针from
    # 后向回溯
    path = []  # 最短路径
    f = nodes[len(nodes) - 1].getFirst()  # 从终点回溯
    while f:
        path.insert(0, f)
        f = f.getFrom()  # 按前驱指针from回溯
    return [v.realWord for v in path]


## 训练n元语法模型
def train_bigram(corpus_path, model_path):
    sents = CorpusLoader.convert2SentenceList(corpus_path)
    for sent in sents:
        for word in sent:
            word.setLabel("n")
    
    maker = NatureDictionaryMaker()
    maker.compute(sents)
    maker.saveTxtTo(model_path)      # 会生成两个统计词频文件

## 加载 n元语法模型
def load_bigram(model_path, sent):
    HanLP.Config.CoreDictionaryPath = model_path + ".txt"  # unigram
    HanLP.Config.BiGramDictionaryPath = model_path + ".ngram.txt"  # bigram
    
    wordnet = generate_wordnet(sent, CoreDictionary.trie)
    print(viterbi(wordnet))



corpus_path = 'my_cws_corpus.txt' # 语料库文件
model_path = 'my_cws_model'       # 模型保存路径
sent = '货币和服务'                 # 需要分词的语句

train_bigram(corpus_path, model_path)
load_bigram(model_path, sent)





HanLP分词器简洁版：

In [ ]:
from pyhanlp import *

ViterbiSegment = SafeJClass('com.hankcs.hanlp.seg.Viterbi.ViterbiSegment')

segment = ViterbiSegment()
sentence = "社会摇摆简称社会摇"
segment.enableCustomDictionary(False)
print("不挂载词典：", segment.seg(sentence))
CustomDictionary.insert("社会摇", "nz 100")
segment.enableCustomDictionary(True)
print("低优先级词典：", segment.seg(sentence))
segment.enableCustomDictionaryForcing(True)
print("高优先级词典：", segment.seg(sentence))

可见，用户词典的高优先级未必是件好事

# 隐马尔可夫模型与序列标注

## 隐马尔可夫模型的训练

案例假设和模型构造

In [10]:
import numpy as np
from pyhanlp import *
from jpype import JArray, JFloat, JInt

to_str = JClass('java.util.Arrays').toString

## 隐马尔可夫模型描述
states = ('Healthy', 'Fever')
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
transition_probability = {
    'Healthy': {'Healthy': 0.7, 'Fever': 0.3},
    'Fever': {'Healthy': 0.4, 'Fever': 0.6},
}
emission_probability = {
    'Healthy': {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
    'Fever': {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
}
observations = ('normal', 'cold', 'dizzy')


def generate_index_map(lables):
    index_label = {}
    label_index = {}
    i = 0
    for l in lables:
        index_label[i] = l
        label_index[l] = i
        i += 1
    return label_index, index_label


states_label_index, states_index_label = generate_index_map(states)
observations_label_index, observations_index_label = generate_index_map(observations)



def convert_map_to_matrix(map, label_index1, label_index2):
    m = np.empty((len(label_index1), len(label_index2)), dtype=float)
    for line in map:
        for col in map[line]:
            m[label_index1[line]][label_index2[col]] = map[line][col]
    return JArray(JFloat, m.ndim)(m.tolist())

def convert_observations_to_index(observations, label_index):
    list = []
    for o in observations:
        list.append(label_index[o])
    return list

def convert_map_to_vector(map, label_index):
    v = np.empty(len(map), dtype=float)
    for e in map:
        v[label_index[e]] = map[e]
    return JArray(JFloat, v.ndim)(v.tolist())  # 将numpy数组转为Java数组


## pi：初始状态概率向量
## A：状态转移概率矩阵
## B：发射概率矩阵
A = convert_map_to_matrix(transition_probability, states_label_index, states_label_index)
B = convert_map_to_matrix(emission_probability, states_label_index, observations_label_index)
observations_index = convert_observations_to_index(observations, observations_label_index)
pi = convert_map_to_vector(start_probability, states_label_index)

FirstOrderHiddenMarkovModel = JClass('com.hankcs.hanlp.model.hmm.FirstOrderHiddenMarkovModel')
given_model = FirstOrderHiddenMarkovModel(pi, A, B)

样本生成算法

In [11]:
## 第一个参数：序列最低长度
## 第二个参数：序列最高长度
## 第三个参数：需要生成的样本数
for O, S in given_model.generate(3, 5, 2):
    print(" ".join((observations_index_label[o] + '/' + states_index_label[s]) for o, s in zip(O, S)))

cold/Fever cold/Fever dizzy/Fever
normal/Healthy cold/Healthy normal/Healthy cold/Healthy


隐马尔可夫模型的训练

In [12]:
trained_model = FirstOrderHiddenMarkovModel()

## 第一个参数：序列最低长度
## 第二个参数：序列最高长度
## 第三个参数：需要生成的样本数
trained_model.train(given_model.generate(3, 10, 100000))
print('新模型与旧模型是否相同：', trained_model.similar(given_model))

新模型与旧模型是否相同： True


## 隐马尔可夫模型的预测

搜索状态序列的维特比算法

In [13]:
pred = JArray(JInt, 1)([0, 0, 0])
prob = given_model.predict(observations_index, pred)
print(" ".join((observations_index_label[o] + '/' + states_index_label[s]) for o, s in
               zip(observations_index, pred)) + " {:.3f}".format(np.math.exp(prob)))

normal/Healthy cold/Healthy dizzy/Fever 0.015
